<a href="https://colab.research.google.com/github/okunkov/park_view/blob/master/Reketa_PROD_v4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Заполни дату
# @markdown Дата начала выписки

date = '2023-09-01'  # @param {type: "date"}

from datetime import datetime, timezone


def string_to_isotime(date_string):
    datetime_object = datetime.fromisoformat(date_string)
    return datetime_object.strftime("%Y-%m-%dT%H:%M:%S.000Z")


# datetime_object = datetime.strptime(date, '%Y-%m-%d')
# iso_date = datetime_object.strftime("%Y-%m-%dT%H:%M:%S.000Z")

DATE_FROM = string_to_isotime(date)
DATE_NOW = datetime.now().strftime("%Y-%m-%dT%H:%M:%S.000Z")
print(f"{DATE_FROM = }\n{DATE_NOW = }")


DATE_FROM = '2023-09-01T00:00:00.000Z'
DATE_NOW = '2023-12-06T14:16:08.000Z'


In [ ]:
#@title Загрузи .xlsx файл
#@markdown

from google.colab import files
uploaded = files.upload()





Saving Верваг111.xls to Верваг111.xls


In [ ]:
# @title Запускай
# @markdown


import requests
import pandas as pd
import re
import io
from time import sleep
# from tables import file


#  DEPRICATED !!!!!!!!!!!!!!!!!!!!!!!!!!
def parse_ticket_number(string_data):
    '''
        Parse ticket number from text
        *** в API к номеру билеты прибавляется код компании - пример 'number': '5556177886145'
    '''
    result = re.search(r"\d{3}\s\d{10} | \d{14}", string_data)
    if result:
        result = result.group().replace(" ", "")
        return result
    else:
        print('Номер билета не найден')
        return None

## TEST
# excel_string = 'Авиабилет МАЦИЯК МАКСИМ ВИКТОРОВИЧ  425 6110231540  Москва - Самара - Москва    28.10.23'
# print(parse_ticket_number(excel_string))


def find_order_clientid_dest(ticket_number, json):
    '''
    Get order id, client id and destination by ticket number
    '''
    result = [None, None, None]

    if not (ticket_number and json):
        print('Не указан номер билета или выгрузука Ракеты, ошибка функции')
        return result

    for element in json:
        if element['type'] == 'avia' \
            and element['avia']['tickets'] \
            and element['avia']['tickets'][0]['number'] == ticket_number:
            result = [element['orderId'], element['avia']['tickets'][0]['passengerId'], element['avia']['toAirportName']]
            return result
        if element['type'] == 'rail' \
            and element['rail']['forward'][0]['tickets'] \
            and element['rail']['forward'][0]['tickets'][0]['number'] == ticket_number:
            result = [element['orderId'], element['rail']['forward'][0]['tickets'][0]['passengerId'], element['rail']['forward'][0]['toStation']]
            return result
        if element['type'] == 'hotel' \
            and element['hotel']['bookingReference'] \
            and element['hotel']['bookingReference'] == ticket_number:
            result = [element['orderId'], element['travellers'][0], None]
            return result

    print(f"Информация о билете ({ticket_number}) не найдена.")
    return result

## TEST
# print(find_order_clientid_dest('5556179106551'))


def find_tabnum_cfo_code_fio(client_id, headers):
    '''
    Найти ЦФО, Код ЦФО, табельный номер + ФИО
    '''
    if client_id:
        client_id = str(client_id)
        profiles_endpoint = 'https://agency.raketa.travel/v2/profiles/' + client_id
        json_data_profiles = requests.get(profiles_endpoint, headers=headers).json()

        fio = " ".join([json_data_profiles['lastName'], json_data_profiles['firstName'], json_data_profiles['middleName']])

        if json_data_profiles['extrafields']:
            return [json_data_profiles['code'], json_data_profiles['costCenter'], json_data_profiles['extrafields'][0]['value'], fio]
        else:
            return [json_data_profiles['code'], json_data_profiles['costCenter'], fio]
    else:
        print('ЦФО, код ЦФО и таб.номер - не найден')
        return [None, None, None, None]
## TEST
# order_id, client_id = '5356397', '1548785'
# print(find_tabnum_cfo_code_fio(client_id))


def find_brand(order_id, headers):
    '''
    Поиска Бренда
    '''
    if order_id:
        order_id = str(order_id)
        brand_endpoint = f'https://agency.raketa.travel/v2/orders/{order_id}'
        json_data_brand = requests.get(brand_endpoint, headers=headers).json()

        if json_data_brand['extrafields']:
            ''' С номером элеманта массива [1] могут быть *фокусы* '''
            return str(json_data_brand['extrafields'][1]['value'])
    else:
        print('Бренд - не найден')
        return None

## TEST
# print(find_brand(order_id))


def find_businesstrip(order_id, headers):
    '''
    Поиск Цель командировки
    '''
    if order_id:
        order_id = str(order_id)
        profiles_endpoint = f'https://agency.raketa.travel/v2/orders/{order_id}/businessTrip'
        json_data_businesstrip = requests.get(profiles_endpoint, headers=headers).json()
        if json_data_businesstrip['purpose']:
            return str(json_data_businesstrip['purpose'])
    else:
        print('Цель командировки - не найдена')
        return None

# TEST
# print(find_businesstrip(5356397))


def validate_ticket(code_ak, ticket_number, service_type):
    '''
    Получение номера билета на основании типа услуги
    '''
    if service_type == "Ж/Д билет" or service_type == "Отельная бронь":
        return str(ticket_number)
    elif service_type == "Авиабилет":
        return "".join([str(code_ak), str(ticket_number)])
    else:
        return None


def main():
    '''
    MAIN FUNCTION
    '''
    filename = list(uploaded.keys())
    print('Excel filename is:', filename)

    df = pd.read_excel(io.BytesIO(uploaded[filename[0]]), header=[0])
    # print(df.columns)
    # print(df.head())

    # VARS
    date_from = DATE_FROM
    token = 'c52a73d26c1a487901da43f1c30bdf37ff98adfae2bdf3574efa10cd4249e55d'
    headers = {'Authorization': f'Bearer {token}', 'accept': 'application/json'}

    services_endpoint = 'https://agency.raketa.travel/v2/services?fromLastUpdate=' + date_from
    json_data_services = requests.get(services_endpoint, headers=headers).json()
    json_last_date = json_data_services[len(json_data_services)-1]['lastUpdate']

    while json_last_date[:10] != DATE_NOW[:10]:
        date_from = string_to_isotime(json_last_date)
        url = "https://agency.raketa.travel/v2/services?fromLastUpdate=" + date_from
        new_request = requests.get(url, headers=headers).json()
        if not new_request: break

        print(f'New request date: {date_from} {len(new_request) = }')

        json_data_services.extend(new_request)
        json_last_date = new_request[len(new_request)-1]['lastUpdate']
        print(f'{json_data_services[len(json_data_services)-1]["lastUpdate"]}')



    for index, row in df.iterrows():

        ''' Печатаем разделитель '''
        print(f"\n\n Строка excel #{index + 2} {'='*50}")

        '''
        Проверяем билет, что строка nan из пандас == float (странный такой нюанс).
        '''
        ticket_number = validate_ticket(row[7], row[8], row[17])


        if ticket_number:
            order_id, client_id, destination = find_order_clientid_dest(ticket_number, json_data_services)
            # print("* OrderID, ClientID, Destination *:", order_id, client_id, destination)
            if order_id:
                tabnum, cfo, cfo_code, fio = find_tabnum_cfo_code_fio(client_id, headers)
                businesstrip = find_businesstrip(order_id, headers)
                brand = find_brand(order_id, headers)

                if len(ticket_number) == 13:
                    ticket_number = ticket_number[3::]

                print(f"{'№ СОБ' : <10} {order_id}\n"
                    f"{'№ билета' : <10} {ticket_number}\n"
                    f"{'ФИО' : <10} {fio}\n"
                    f"{'№ Клиента' : <10} {client_id}\n" # технические данные - не требуются для .xls
                    f"{'Маршрут' : <10} {destination}\n"
                    f"{'Таб.номер' : <10} {tabnum}\n"
                    f"{'ЦФО' : <10} {cfo}\n"
                    f"{'Код ЦФО' : <10} {cfo_code}\n"
                    f"{'Цель ком.' : <10} {businesstrip}\n"
                    f"{'Бренд' : <10} {brand}\n")


                '''
                Add collected data to DataFrame
                '''
                print("* Записываем данные в DataFrame *")
                df.at[index, 'Номер заказа в СОБ'] = order_id
                df.at[index, 'ФИО пассажира'] = fio
                df.at[index, 'Место командировки'] = destination
                # df.at[index, 'Номер билета'] = ticket_number
                df.at[index, 'Табельный номер'] = tabnum
                df.at[index, 'ЦФО'] = cfo
                df.at[index, 'Код ЦФО'] = cfo_code
                df.at[index, 'Цель командировки'] = businesstrip
                df.at[index, 'Бренд'] = brand

                '''
                Собираем строку "Наименование услуги" из того, что записали внесли в DataFrame нового файла
                Если "код АК" пустой, то не включаем его в тарифицируемую позицию (billing_item)
                '''
                if row[17] == "Ж/Д билет":
                    billing_item = f"{df.at [index,'Тип услуги']} {fio} {df.at [index,'номер билета']} {df.at [index,'Место командировки']} {df.at [index,'Дата начала командировки (вылета)']} {df.at [index,'Дата окончания командировки (прилета)']}"
                elif row[17] == "Авиабилет":
                    billing_item = f"{df.at [index,'Тип услуги']} {fio} {df.at [index,'код АК']} {df.at [index,'номер билета']} {df.at [index,'Место командировки']} {df.at [index,'Дата начала командировки (вылета)']} {df.at [index,'Дата окончания командировки (прилета)']}"
                else:
                    billing_item = f"{df.at [index,'Тип услуги']} {fio} {df.at [index,'номер билета']}"

                print(billing_item)
                df.at[index, 'Наименование услуги'] = billing_item

    '''
    Запись измененного DataFrame в файл Excel
    '''
    df.to_excel('Petrovax_two_week_report.xlsx', index=False)


if __name__ == '__main__':
    main()


Excel filename is: ['Верваг111.xls']
New request date: 2023-09-13T08:26:51.000Z len(new_request) = 100
2023-09-25T11:33:10+03:00
New request date: 2023-09-25T11:33:10.000Z len(new_request) = 100
2023-10-05T00:12:31+03:00
New request date: 2023-10-05T00:12:31.000Z len(new_request) = 102
2023-10-17T19:00:02+03:00
New request date: 2023-10-17T19:00:02.000Z len(new_request) = 100
2023-10-26T19:00:03+03:00
New request date: 2023-10-26T19:00:03.000Z len(new_request) = 100
2023-11-07T10:23:02+03:00
New request date: 2023-11-07T10:23:02.000Z len(new_request) = 100
2023-11-09T11:24:39+03:00
New request date: 2023-11-09T11:24:39.000Z len(new_request) = 100
2023-11-17T09:42:04+03:00
New request date: 2023-11-17T09:42:04.000Z len(new_request) = 100
2023-11-26T13:06:31+03:00
New request date: 2023-11-26T13:06:31.000Z len(new_request) = 100
2023-12-01T16:20:04+03:00
New request date: 2023-12-01T16:20:04.000Z len(new_request) = 46
2023-12-06T15:42:17+03:00


 Строка excel #2 =========================